## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from pprint import pprint

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mount Gambier,AU,-37.8333,140.7667,18.95,55,75,9.77,broken clouds
1,1,Albany,US,42.6001,-73.9662,-7.46,67,9,0.96,clear sky
2,2,Barrow,US,71.2906,-156.7887,-25.99,76,0,3.60,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,5.81,81,75,10.29,broken clouds
4,4,Arman,RU,59.7000,150.1667,-14.39,77,100,6.39,overcast clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 15
What is the maximum temperature you would like for your trip? 50


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mount Gambier,AU,-37.8333,140.7667,18.95,55,75,9.77,broken clouds
5,5,San Quintin,MX,30.4833,-115.9500,15.45,68,100,5.32,overcast clouds
6,6,Hobart,AU,-42.8794,147.3294,28.92,44,40,1.79,scattered clouds
7,7,Sao Miguel Do Iguacu,BR,-25.3481,-54.2378,27.56,63,21,1.42,few clouds
8,8,Mataura,NZ,-46.1927,168.8643,24.15,49,100,0.83,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,25.64,67,99,2.38,overcast clouds
11,11,Bluff,NZ,-46.6000,168.3333,24.33,78,100,2.69,overcast clouds
12,12,New Norfolk,AU,-42.7826,147.0587,28.72,39,27,1.79,scattered clouds
13,13,Souillac,MU,-20.5167,57.5167,26.31,78,75,8.23,broken clouds
19,19,Samarai,PG,-10.6167,150.6667,30.07,66,100,5.62,overcast clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                215
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country","Lat","Lng", "Max Temp","Humidity","Cloudiness", "Wind Speed","Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,Mount Gambier,AU,-37.8333,140.7667,18.95,55,75,9.77,broken clouds,
5,San Quintin,MX,30.4833,-115.9500,15.45,68,100,5.32,overcast clouds,
6,Hobart,AU,-42.8794,147.3294,28.92,44,40,1.79,scattered clouds,
7,Sao Miguel Do Iguacu,BR,-25.3481,-54.2378,27.56,63,21,1.42,few clouds,
8,Mataura,NZ,-46.1927,168.8643,24.15,49,100,0.83,overcast clouds,
9,Rikitea,PF,-23.1203,-134.9692,25.64,67,99,2.38,overcast clouds,
11,Bluff,NZ,-46.6000,168.3333,24.33,78,100,2.69,overcast clouds,
12,New Norfolk,AU,-42.7826,147.0587,28.72,39,27,1.79,scattered clouds,
13,Souillac,MU,-20.5167,57.5167,26.31,78,75,8.23,broken clouds,
19,Samarai,PG,-10.6167,150.6667,30.07,66,100,5.62,overcast clouds,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #pprint(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #print(hotels["results"][0]["name"])
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
#print(hotel_df)   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,Mount Gambier,AU,-37.8333,140.7667,18.95,55,75,9.77,broken clouds,The Commodore
5,San Quintin,MX,30.4833,-115.9500,15.45,68,100,5.32,overcast clouds,Don Eddie's Landing
6,Hobart,AU,-42.8794,147.3294,28.92,44,40,1.79,scattered clouds,Mantra on Collins Hobart
7,Sao Miguel Do Iguacu,BR,-25.3481,-54.2378,27.56,63,21,1.42,few clouds,Hotel Itaguaçu
8,Mataura,NZ,-46.1927,168.8643,24.15,49,100,0.83,overcast clouds,Ellie's Villa
...,...,...,...,...,...,...,...,...,...,...
403,Providencia,MX,28.7167,-111.5833,24.92,41,8,4.36,clear sky,
405,Kelo,TD,9.3086,15.8066,19.01,19,18,2.17,few clouds,Plantation de Kelo
406,Kaabong,UG,3.4836,34.1492,19.34,45,99,0.43,overcast clouds,Rock Motel Ltd
407,Naifaru,MV,5.4444,73.3657,27.82,74,93,6.07,overcast clouds,Naifaru Residence Guest House


In [40]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['City'].astype(bool)]
hotel_df = hotel_df[hotel_df['Country'].astype(bool)]
hotel_df = hotel_df[hotel_df['Lat'].astype(bool)]
hotel_df = hotel_df[hotel_df['Lng'].astype(bool)]
hotel_df = hotel_df[hotel_df['Max Temp'].astype(bool)]
hotel_df = hotel_df[hotel_df['Humidity'].astype(bool)]
hotel_df = hotel_df[hotel_df['Cloudiness'].astype(bool)]
hotel_df = hotel_df[hotel_df['Wind Speed'].astype(bool)]
hotel_df = hotel_df[hotel_df['Current Description'].astype(bool)]
hotel_df = hotel_df[hotel_df['Hotel Name'].astype(bool)]
clean_hotel_df = hotel_df.dropna(how='any')

In [41]:
clean_hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Hotel Name
0,Mount Gambier,AU,-37.8333,140.7667,18.95,55,75,9.77,broken clouds,The Commodore
5,San Quintin,MX,30.4833,-115.9500,15.45,68,100,5.32,overcast clouds,Don Eddie's Landing
6,Hobart,AU,-42.8794,147.3294,28.92,44,40,1.79,scattered clouds,Mantra on Collins Hobart
7,Sao Miguel Do Iguacu,BR,-25.3481,-54.2378,27.56,63,21,1.42,few clouds,Hotel Itaguaçu
8,Mataura,NZ,-46.1927,168.8643,24.15,49,100,0.83,overcast clouds,Ellie's Villa
...,...,...,...,...,...,...,...,...,...,...
399,San Juan,AR,-31.5375,-68.5364,28.40,24,49,3.25,scattered clouds,Hotel Provincial
405,Kelo,TD,9.3086,15.8066,19.01,19,18,2.17,few clouds,Plantation de Kelo
406,Kaabong,UG,3.4836,34.1492,19.34,45,99,0.43,overcast clouds,Rock Motel Ltd
407,Naifaru,MV,5.4444,73.3657,27.82,74,93,6.07,overcast clouds,Naifaru Residence Guest House


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))